In [1]:
from data_pipeline_v3 import DataGen
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.utils import class_weight

Set up the dataset from our generator:

In [2]:
num_epochs = 50

In [3]:
def _fixup_shape(x, y):
  x.set_shape([None, 259, 128]) # n, h, w, c
  y.set_shape([None]) # n, nb_classes
  return x, y

batch_size = 16
tracks = pd.read_csv('./data/processed_genres_mel.csv')

# Set up train/test split
all_idxs = list(range(len(tracks)))
np.random.shuffle(all_idxs)
num_train = np.floor(len(all_idxs) * 0.8).astype(np.int32)
train_idxs = all_idxs[:num_train]
test_idxs = all_idxs[num_train:]

# Set up generator processing function
gen = DataGen(tracks, batch_size=batch_size)

# Set up train data
train_dataset = tf.data.Dataset.from_generator(lambda: train_idxs, tf.uint16)
train_dataset = train_dataset.shuffle(buffer_size=len(train_idxs),
                                      seed=0, reshuffle_each_iteration=True)
train_dataset = train_dataset.map(lambda i: tf.py_function(func=gen.get_sample, 
                                                           inp=[i], 
                                                           Tout=[tf.float32,
                                                                 tf.int32]
                                                          ), 
                                                          num_parallel_calls=tf.data.AUTOTUNE)
train_dataset = train_dataset.repeat(num_epochs)
train_dataset = train_dataset.batch(batch_size).map(_fixup_shape)
train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)

# Set up test data
test_dataset = tf.data.Dataset.from_generator(lambda: test_idxs, tf.uint16)
test_dataset = test_dataset.shuffle(buffer_size=len(test_idxs),
                                      seed=0, reshuffle_each_iteration=True)
test_dataset = test_dataset.map(lambda i: tf.py_function(func=gen.get_sample, 
                                                           inp=[i], 
                                                           Tout=[tf.float32,
                                                                 tf.int32]
                                                          ), 
                                                          num_parallel_calls=tf.data.AUTOTUNE)
test_dataset = test_dataset.repeat(num_epochs)
test_dataset = test_dataset.batch(batch_size).map(_fixup_shape)
test_dataset = test_dataset.prefetch(tf.data.AUTOTUNE)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [4]:
next(iter(train_dataset))

(<tf.Tensor: shape=(16, 259, 128), dtype=float32, numpy=
 array([[[0.7137642 , 0.78672   , 0.8449787 , ..., 0.        ,
          0.        , 0.        ],
         [0.7178279 , 0.78846943, 0.84625506, ..., 0.        ,
          0.        , 0.        ],
         [0.68459123, 0.77611125, 0.83741474, ..., 0.        ,
          0.        , 0.        ],
         ...,
         [0.7334004 , 0.80620414, 0.8644324 , ..., 0.        ,
          0.        , 0.        ],
         [0.7251082 , 0.8026834 , 0.86181706, ..., 0.        ,
          0.        , 0.        ],
         [0.7177766 , 0.8008223 , 0.8608847 , ..., 0.        ,
          0.        , 0.        ]],
 
        [[0.7382328 , 0.7213874 , 0.6793546 , ..., 0.        ,
          0.        , 0.        ],
         [0.7535432 , 0.74835074, 0.7509733 , ..., 0.        ,
          0.        , 0.        ],
         [0.73325455, 0.71427023, 0.65548503, ..., 0.        ,
          0.        , 0.        ],
         ...,
         [0.83052325, 0.818075

Compute the class weights for balancing:

In [5]:
genres = np.array(tracks['parent_genre_id'])
class_weights = class_weight.compute_class_weight(class_weight='balanced',
                                                  classes=np.unique(genres),
                                                  y=genres)

class_weights = dict(enumerate(class_weights))

class_weights

{0: 0.7109481971702419,
 1: 1.31339586846543,
 2: 0.21964008742244784,
 3: 1.0261445981554678,
 4: 0.6932298620382733,
 5: 4.056477864583333,
 6: 0.24701673009831907,
 7: 1.5499378109452737,
 8: 10.114853896103897,
 9: 21.04983108108108,
 10: 13.200741525423728,
 11: 8.751053370786517,
 12: 2.5328252032520324,
 13: 3.2051183127572016,
 14: 1.1555545252225519,
 15: 74.17559523809524}

Build and train the model:

In [6]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(259, dropout=0.2, recurrent_dropout=0.2), input_shape=(259, 128)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(256, activation="relu"),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(32, activation="relu"),  
    tf.keras.layers.Dense(16)
])
    
model.summary()
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=["accuracy"], optimizer='adam')

history = model.fit(x=train_dataset, epochs=50,
                    validation_data=test_dataset, class_weight=class_weights,
                    steps_per_epoch=len(train_idxs) // batch_size)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 518)              803936    
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 518)               0         
                                                                 
 dense (Dense)               (None, 256)               132864    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                        